<a href="https://colab.research.google.com/github/shiragelb/NCC-Statistical-Reports/blob/main/Table_Extraction_Validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install pandoc
!pip install pypandoc
!pip install python-docx
!pip install docx2txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc-data
Suggested packages:
  texlive-latex-recommended texlive-xetex texlive-luatex pandoc-citeproc
  texlive-latex-extra context wkhtmltopdf librsvg2-bin groff ghc nodejs php
  python ruby libjs-mathjax libjs-katex citation-style-language-styles
The following NEW packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc
  pandoc-data
0 upgraded, 4 newly installed, 0 to remove and 35 not upgraded.
Need to get 20.6 MB of archives.
After this operation, 156 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [115 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm-extensions0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [25.1 kB

In [ ]:
#extracting folders id from shared memory

from google.colab import auth
from googleapiclient.discovery import build
import pandas as pd

# Authenticate
auth.authenticate_user()
drive_service = build('drive', 'v3')

folder_id = "1e0eA-AIsz_BSwVHOppJMXECX42hBfG4J"

def list_all_files_in_folder_recursive(parent_id, parent_path=""):
    """Recursively list all docx files in a folder and subfolders"""
    all_files = []

    query = f"'{parent_id}' in parents and trashed=false"
    page_token = None

    while True:
        response = drive_service.files().list(
            q=query,
            spaces='drive',
            fields='nextPageToken, files(id, name, mimeType)',
            pageToken=page_token
        ).execute()

        for item in response.get('files', []):
            item_path = f"{parent_path}/{item['name']}" if parent_path else item['name']

            if item['mimeType'] == 'application/vnd.google-apps.folder':
                # Recurse into subfolder
                all_files.extend(list_all_files_in_folder_recursive(item['id'], item_path))
            elif item['name'].endswith('.docx'):
                # Only add docx files
                all_files.append({
                    "file_name": item['name'],
                    "file_path": item_path,
                    "file_id": item['id'],
                    "file_url": f"https://drive.google.com/file/d/{item['id']}/view?usp=sharing"
                })

        page_token = response.get('nextPageToken', None)
        if page_token is None:
            break

    return all_files

# Run the recursive function
files_list = list_all_files_in_folder_recursive(folder_id)

# Convert to DataFrame
df_files = pd.DataFrame(files_list)
##df_files.head()

##getting local id of the file
!pip install python-docx
import os
from googleapiclient.http import MediaIoBaseDownload
import io

def get_chapter_file(chapter_num: int, year: str, download_dir="/content") -> str:
    """
    Download a chapter file for a given year from Google Drive and return its local path.

    Args:
        chapter_num (int): The chapter number (e.g., 1, 2, ..., 14).
        year (str): The year folder name.
        download_dir (str): Local folder to save the file. Defaults to /content in Colab.

    Returns:
        str: Local path to the downloaded file, or None if not found.
    """
    # Normalize chapter filename (01, 02, ...)
    chapter_str = f"{chapter_num:02}"

    # Find matching file in df_files
    match = df_files[
        (df_files["file_name"].str.contains(chapter_str)) &
        (df_files["file_path"].str.contains(f"{year}/"))
    ]

    if match.empty:
        print(f"❌ No file found for chapter {chapter_str} in year {year}")
        return None

    file_id = match.iloc[0]["file_id"]
    file_name = match.iloc[0]["file_name"]
    local_path = os.path.join(download_dir, year, file_name)

    # Ensure year directory exists locally
    os.makedirs(os.path.dirname(local_path), exist_ok=True)

    # Download the file from Google Drive
    request = drive_service.files().get_media(fileId=file_id)
    fh = io.FileIO(local_path, "wb")
    downloader = MediaIoBaseDownload(fh, request)

    done = False
    while not done:
        status, done = downloader.next_chunk()
        if status:
            print(f"⬇️ Download {int(status.progress() * 100)}%.")

    print(f"✅ Saved to {local_path}")
    return local_path

from docx import Document

# After downloading the file
# sanity check
path = get_chapter_file(14, "2015")
print("Local file:", path)

if path and path.endswith(".docx"):
    doc = Document(path)
    full_text = "\n".join([p.text for p in doc.paragraphs if p.text.strip() != ""])

    # Split into sentences (naive split by period/full stop)
    sentences = full_text.replace("\n", " ").split(".")

    # Print first few sentences
    preview_count = 3
    preview_sentences = [s.strip() for s in sentences if s.strip()][:preview_count]
    print("\nPreview of content:")
    for i, s in enumerate(preview_sentences, 1):
        print(f"{i}. {s}.")


#a code that downloads the files to a directory called content/reports
import os
import io
from googleapiclient.http import MediaIoBaseDownload

def download_all_chapters(download_dir="/content/reports", years=range(2001, 2025), chapters=range(1, 16)):
    """
    Downloads all chapters for all years to local environment.

    Returns:
        dict: {year: {chapter_number: local_path}}
    """
    all_paths = {}

    for year in years:
        year_str = str(year)
        all_paths[year_str] = {}

        for chapter in chapters:
            chapter_str = f"{chapter:02}"

            # Find matching file in df_files
            match = df_files[
                (df_files["file_name"].str.contains(chapter_str)) &
                (df_files["file_path"].str.contains(f"{year_str}/"))
            ]

            if match.empty:
                print(f"⚠️ Chapter {chapter_str} not found for year {year_str}")
                continue

            file_id = match.iloc[0]["file_id"]
            file_name = match.iloc[0]["file_name"]
            local_path = os.path.join(download_dir, year_str, f"{chapter_str}_{file_name}")

            # Ensure folder exists
            os.makedirs(os.path.dirname(local_path), exist_ok=True)

            # Download file
            request = drive_service.files().get_media(fileId=file_id)
            fh = io.FileIO(local_path, "wb")
            downloader = MediaIoBaseDownload(fh, request)

            done = False
            while not done:
                status, done = downloader.next_chunk()
                if status:
                    print(f"⬇️ Download {int(status.progress() * 100)}% for {file_name}")

            print(f"✅ Saved {file_name} to {local_path}")
            all_paths[year_str][chapter] = local_path

    return all_paths

# Donwloads all docx files - ignores doc or pdf
download_all_chapters()

In [ ]:
import os
import json
import pandas as pd
from docx import Document

def extract_tables_from_reports(base_dir="/content/reports", out_dir="/content/tables"):
    os.makedirs(out_dir, exist_ok=True)
    summary = {}
    colnames_map = {}

    for year in range(2001, 2025):  # adjust later
        print(year)
        year_path = os.path.join(base_dir, str(year))
        if not os.path.isdir(year_path):
            continue

        for fname in os.listdir(year_path):
            if not (fname.endswith(".docx") or fname.endswith(".doc")):
                continue
            if fname.endswith(".doc"):
                continue

            chapter = fname.split("_")[0]
            fpath = os.path.join(year_path, fname)

            try:
                doc = Document(fpath)
            except Exception as e:
                print(f"skip {fpath}: {e}")
                continue

            serial = 1
            # find all tables with names containing "לוח"
            for i, table in enumerate(doc.tables):
                table_name = ""  # default

                # check the first row cells for "לוח"
                if len(table.rows) > 0:
                    for cell in table.rows[0].cells:
                        if "לוח" in cell.text and "תרשים" not in cell.text:  # extra check
                            table_name = cell.text.strip()
                            break

                    if not table_name:  # skip if not a valid table
                        continue

                    # convert table to dataframe
                    data = [[cell.text.strip() for cell in row.cells] for row in table.rows]
                    df = pd.DataFrame(data)

                    identifier = f"{serial}_{chapter}_{year}"

                    # record mapping for JSON
                    if len(df) > 0:
                        summary[identifier] = " ".join(df.iloc[0].astype(str).tolist())
                    else:
                        continue

                    if len(df) > 1:
                        colnames_map[identifier] = df.iloc[1].astype(str).tolist()
                    else:
                        colnames_map[identifier] = []

                    # save CSV
                    save_dir = os.path.join(out_dir, str(year), chapter)
                    os.makedirs(save_dir, exist_ok=True)
                    save_path = os.path.join(save_dir, f"{identifier}.csv")
                    df.to_csv(save_path, index=False, encoding="utf-8-sig")

                    serial += 1

    # write summary JSON
    with open(os.path.join(out_dir, "tables_summary.json"), "w", encoding="utf-8") as f:
        json.dump(summary, f, ensure_ascii=False, indent=2)

    with open(os.path.join(out_dir, "tables_columns.json"), "w", encoding="utf-8") as f:
        json.dump(colnames_map, f, ensure_ascii=False, indent=2)


extract_tables_from_reports()

# Table Extraction Validation Plan

## Project Overview
Validation strategy for Hebrew table extraction workflow processing ~24 years × 14-15 chapters × 10-20 tables per chapter. Focus: ensuring complete extraction of all "לוח" tables with proper data integrity.

## Approved Validation Strategy

### Phase 1: Document-Level Extraction Validation (Automated)

#### Step 1: Table Position Mapping
- **Purpose**: Track document position of each extracted table to detect gaps
- **Method**: Record paragraph/table index in original document for each extraction
- **Output**: Position mapping file showing extraction sequence
- **Rationale**: Identifies if tables were skipped during document traversal

#### Step 2: Serial Number Sequence Validation  
- **Purpose**: Verify consecutive serial numbering within each document
- **Method**: Check CSV filenames against `tables_summary.json` for gaps in serial sequence
- **Expected Pattern**: 1, 2, 3, 4... (no missing numbers per document)
- **Output**: List of documents with missing serial numbers
- **Rationale**: Missing serials indicate extraction failures

### Phase 2: CSV Quality & Integrity Checks (Automated)

#### Step 1: File Completeness Validator
- **Purpose**: Ensure all expected files exist with correct naming
- **Method**: Validate `serial_chapter_year.csv` naming convention
- **Cross-check**: Verify files exist for all entries in `tables_summary.json`
- **Output**: List of missing or misnamed files

#### Step 2: Data Integrity Checker
- **Truncation Detection**: Flag tables with suspiciously low row/column counts
- **Empty File Detection**: Identify completely empty CSV files
- **Encoding Validation**: Check for encoding issues or malformed data
- **Header Validation**: Verify first row contains "לוח" as expected
- **Output**: Integrity issues report with specific file problems

#### Step 3: Content Pattern Validator
- **Purpose**: Ensure extracted data maintains expected structure
- **Method**: Validate Hebrew text presence and basic table patterns
- **Checks**: Proper CSV format, readable Hebrew characters
- **Output**: Pattern validation report

### Phase 3: Statistical Anomaly Detection (Automated)

#### Step 1: Conservative Cross-Chapter Analysis
- **Purpose**: Flag extreme deviations in table counts
- **Method**:
  - Track historical patterns for each chapter name
  - Flag only deviations >50% from chapter's historical range
  - Avoid false positives from legitimate year-to-year variations
- **Output**: Flagged chapters list for manual TOC verification
- **Rationale**: Focus manual effort on most suspicious cases

#### Step 2: Table Size Distribution Analysis
- **Purpose**: Detect extraction errors through size anomalies
- **Method**: Identify unusually small (1-2 rows) or extremely large tables
- **Thresholds**: Statistical outliers in row/column distributions
- **Output**: Size anomaly report for investigation

#### Step 3: Chapter Completeness Overview
- **Purpose**: Visual pattern recognition for missing extractions
- **Method**: Generate summary matrix showing table counts per chapter-year
- **Output**: Heat map or tabular view for visual inspection
- **Usage**: Quick identification of suspicious patterns

### Phase 4: Targeted Manual Verification (Semi-Automated)

#### Step 1: High-Risk Sample Generator
- **Purpose**: Prioritize manual verification effort
- **Method**: Generate ranked list of documents flagged by automated checks
- **Priority Factors**: Multiple flags, extreme deviations, historical anomalies
- **Output**: Prioritized verification queue for manual TOC checking

#### Step 2: Stratified Random Sampling
- **Purpose**: Quality assurance through representative sampling
- **Method**: Random selection across years/chapters for spot-checking
- **Sample Size**: 5-10% of total documents, stratified by year
- **Process**: Manual comparison of extraction results against original documents
- **Output**: Sampling validation report

## Implementation Sequence

1. **Phase 2** (CSV validation) - Quick automated quality checks
2. **Phase 1** (document-level validation) - Core extraction completeness
3. **Phase 3** (statistical analysis) - Pattern-based anomaly detection  
4. **Phase 4** (manual verification) - Targeted human validation

## Success Criteria

- **Completeness**: No missing serial numbers within documents
- **Integrity**: All CSV files readable with expected Hebrew content
- **Quality**: <5% of extractions flagged as anomalous
- **Validation Coverage**: 100% automated checking + 5-10% manual sampling

## Next Steps

Execute Phase 2 first to establish baseline data quality, then proceed through phases sequentially. Each phase generates specific action items for investigation or correction.

In [ ]:
import os
import json
from pathlib import Path

def validate_file_completeness(tables_dir="/content/tables"):
    """
    Validate that all CSV files referenced in tables_summary.json exist
    and follow the expected naming convention.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    # Check if summary file exists
    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load summary data
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"📊 Checking {len(summary)} table entries...")

    missing_files = []
    naming_issues = []
    found_files = []

    for identifier, table_name in summary.items():
        # Parse identifier: serial_chapter_year
        try:
            parts = identifier.split("_")
            if len(parts) != 3:
                naming_issues.append(f"Invalid identifier format: {identifier}")
                continue

            serial, chapter, year = parts

            # Construct expected file path
            expected_path = os.path.join(tables_dir, year, chapter, f"{identifier}.csv")

            if os.path.exists(expected_path):
                found_files.append(expected_path)
            else:
                missing_files.append({
                    "identifier": identifier,
                    "expected_path": expected_path,
                    "table_name": table_name
                })

        except Exception as e:
            naming_issues.append(f"Error parsing identifier {identifier}: {e}")

    # Generate report
    print("\n" + "="*60)
    print("📋 FILE COMPLETENESS VALIDATION REPORT")
    print("="*60)

    print(f"✅ Files found: {len(found_files)}")
    print(f"❌ Files missing: {len(missing_files)}")
    print(f"⚠️  Naming issues: {len(naming_issues)}")

    if missing_files:
        print(f"\n🔍 MISSING FILES ({len(missing_files)}):")
        for item in missing_files[:10]:  # Show first 10
            print(f"  • {item['identifier']} → {item['expected_path']}")
            print(f"    Table: {item['table_name'][:50]}...")
        if len(missing_files) > 10:
            print(f"  ... and {len(missing_files) - 10} more")

    if naming_issues:
        print(f"\n⚠️  NAMING ISSUES ({len(naming_issues)}):")
        for issue in naming_issues[:5]:  # Show first 5
            print(f"  • {issue}")
        if len(naming_issues) > 5:
            print(f"  ... and {len(naming_issues) - 5} more")

    # Calculate success rate
    total_expected = len(summary)
    success_rate = (len(found_files) / total_expected) * 100 if total_expected > 0 else 0

    print(f"\n📈 SUCCESS RATE: {success_rate:.1f}% ({len(found_files)}/{total_expected})")

    # Save detailed report
    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_file_completeness.json")
    report = {
        "summary": {
            "total_expected": total_expected,
            "files_found": len(found_files),
            "files_missing": len(missing_files),
            "naming_issues": len(naming_issues),
            "success_rate": success_rate
        },
        "missing_files": missing_files,
        "naming_issues": naming_issues,
        "found_files": found_files
    }

    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed report saved to: {report_path}")

    return success_rate > 95  # Consider success if >95% files found

if __name__ == "__main__":
    # Run validation
    is_valid = validate_file_completeness()

    if is_valid:
        print("\n✅ File completeness validation PASSED")
    else:
        print("\n❌ File completeness validation FAILED - review missing files")

📊 Checking 6732 table entries...

📋 FILE COMPLETENESS VALIDATION REPORT
✅ Files found: 6732
❌ Files missing: 0
⚠️  Naming issues: 0

📈 SUCCESS RATE: 100.0% (6732/6732)

💾 Detailed report saved to: /content/tables/validations/validation_file_completeness.json

✅ File completeness validation PASSED


In [ ]:
import os
import json
import pandas as pd
from pathlib import Path
import csv

def check_data_integrity(tables_dir="/content/tables"):
    """
    Check CSV files for truncation, empty files, encoding issues,
    and validate that first row contains "לוח" as expected.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load summary data
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"🔍 Checking data integrity for {len(summary)} CSV files...")

    issues = {
        "empty_files": [],
        "truncated_tables": [],
        "encoding_errors": [],
        "missing_luach": [],
        "malformed_csv": [],
        "suspicious_size": []
    }

    processed = 0
    valid_files = 0

    for identifier, table_name in summary.items():
        try:
            # Parse identifier and construct path
            parts = identifier.split("_")
            if len(parts) != 3:
                continue

            serial, chapter, year = parts
            csv_path = os.path.join(tables_dir, year, chapter, f"{identifier}.csv")

            if not os.path.exists(csv_path):
                continue  # Already handled by completeness validator

            processed += 1
            file_issues = []

            # Check file size
            file_size = os.path.getsize(csv_path)
            if file_size == 0:
                issues["empty_files"].append({
                    "identifier": identifier,
                    "path": csv_path,
                    "table_name": table_name
                })
                continue

            # Try to read the CSV
            try:
                # First try with utf-8-sig (Excel format)
                df = pd.read_csv(csv_path, encoding='utf-8-sig')

                # Check if completely empty
                if df.empty or len(df) == 0:
                    issues["empty_files"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "table_name": table_name,
                        "reason": "DataFrame is empty"
                    })
                    continue

                # Check for suspicious dimensions
                rows, cols = df.shape
                if rows < 2:  # Should have at least header + 1 data row
                    issues["truncated_tables"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "rows": rows,
                        "cols": cols,
                        "reason": "Too few rows (possible truncation)"
                    })

                if cols < 2:  # Tables should have at least 2 columns
                    issues["suspicious_size"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "rows": rows,
                        "cols": cols,
                        "reason": "Too few columns"
                    })

                # Check for extremely large tables (possible extraction error)
                if rows > 1000 or cols > 50:
                    issues["suspicious_size"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "rows": rows,
                        "cols": cols,
                        "reason": "Unusually large table"
                    })

                # Check first row for "לוח"
                has_luach = False
                if len(df) > 0:
                    first_row = df.iloc[0]
                    for cell in first_row:
                        if isinstance(cell, str) and "לוח" in cell:
                            has_luach = True
                            break

                if not has_luach:
                    issues["missing_luach"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "first_row_sample": str(first_row.iloc[0])[:50] if len(first_row) > 0 else "N/A",
                        "table_name": table_name
                    })

                # If we got here, file is basically valid
                if not file_issues:
                    valid_files += 1

            except UnicodeDecodeError as e:
                issues["encoding_errors"].append({
                    "identifier": identifier,
                    "path": csv_path,
                    "error": str(e)
                })
            except pd.errors.EmptyDataError:
                issues["empty_files"].append({
                    "identifier": identifier,
                    "path": csv_path,
                    "reason": "Empty CSV file"
                })
            except Exception as e:
                issues["malformed_csv"].append({
                    "identifier": identifier,
                    "path": csv_path,
                    "error": str(e)
                })

        except Exception as e:
            print(f"⚠️  Error processing {identifier}: {e}")

    # Generate report
    print("\n" + "="*60)
    print("🔍 DATA INTEGRITY VALIDATION REPORT")
    print("="*60)

    total_issues = sum(len(issue_list) for issue_list in issues.values())

    print(f"📊 Files processed: {processed}")
    print(f"✅ Valid files: {valid_files}")
    print(f"❌ Files with issues: {total_issues}")

    for issue_type, issue_list in issues.items():
        if issue_list:
            print(f"\n🚨 {issue_type.replace('_', ' ').upper()} ({len(issue_list)}):")
            for item in issue_list[:3]:  # Show first 3
                if issue_type == "empty_files":
                    print(f"  • {item['identifier']}: {item.get('reason', 'Empty file')}")
                elif issue_type == "truncated_tables":
                    print(f"  • {item['identifier']}: {item['rows']}x{item['cols']} - {item['reason']}")
                elif issue_type == "suspicious_size":
                    print(f"  • {item['identifier']}: {item['rows']}x{item['cols']} - {item['reason']}")
                elif issue_type == "missing_luach":
                    print(f"  • {item['identifier']}: First cell: '{item['first_row_sample']}'")
                elif issue_type == "encoding_errors":
                    print(f"  • {item['identifier']}: {item['error']}")
                elif issue_type == "malformed_csv":
                    print(f"  • {item['identifier']}: {item['error']}")

            if len(issue_list) > 3:
                print(f"  ... and {len(issue_list) - 3} more")

    # Calculate integrity score
    integrity_score = (valid_files / processed) * 100 if processed > 0 else 0

    print(f"\n📈 INTEGRITY SCORE: {integrity_score:.1f}% ({valid_files}/{processed})")

    # Save detailed report
    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_data_integrity.json")
    report = {
        "summary": {
            "files_processed": processed,
            "valid_files": valid_files,
            "total_issues": total_issues,
            "integrity_score": integrity_score
        },
        "issues": issues
    }

    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed report saved to: {report_path}")

    return integrity_score > 90  # Consider success if >90% files are valid

if __name__ == "__main__":
    # Run integrity check
    is_valid = check_data_integrity()

    if is_valid:
        print("\n✅ Data integrity validation PASSED")
    else:
        print("\n❌ Data integrity validation FAILED - review integrity issues")

🔍 Checking data integrity for 6732 CSV files...

🔍 DATA INTEGRITY VALIDATION REPORT
📊 Files processed: 6732
✅ Valid files: 6732
❌ Files with issues: 451

🚨 TRUNCATED TABLES (255):
  • 1_07_2001: 1x1 - Too few rows (possible truncation)
  • 8_14_2001: 1x1 - Too few rows (possible truncation)
  • 1_07_2002: 1x1 - Too few rows (possible truncation)
  ... and 252 more

🚨 SUSPICIOUS SIZE (196):
  • 1_07_2001: 1x1 - Too few columns
  • 8_14_2001: 1x1 - Too few columns
  • 1_07_2002: 1x1 - Too few columns
  ... and 193 more

📈 INTEGRITY SCORE: 100.0% (6732/6732)

💾 Detailed report saved to: /content/tables/validations/validation_data_integrity.json

✅ Data integrity validation PASSED


In [ ]:
import os
import json
import pandas as pd
import re
from pathlib import Path

def validate_content_patterns(tables_dir="/content/tables"):
    """
    Validate that extracted tables contain Hebrew text and proper table structure patterns.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load summary data
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"🔤 Validating content patterns for {len(summary)} CSV files...")

    # Hebrew character range
    hebrew_pattern = re.compile(r'[\u0590-\u05FF]')

    issues = {
        "no_hebrew_content": [],
        "all_empty_cells": [],
        "no_table_structure": [],
        "suspicious_content": []
    }

    processed = 0
    valid_files = 0
    hebrew_stats = {"files_with_hebrew": 0, "total_hebrew_chars": 0}

    for identifier, table_name in summary.items():
        try:
            # Parse identifier and construct path
            parts = identifier.split("_")
            if len(parts) != 3:
                continue

            serial, chapter, year = parts
            csv_path = os.path.join(tables_dir, year, chapter, f"{identifier}.csv")

            if not os.path.exists(csv_path):
                continue  # Already handled by previous validators

            processed += 1
            file_valid = True

            # Read CSV
            try:
                df = pd.read_csv(csv_path, encoding='utf-8-sig')

                if df.empty:
                    continue  # Already handled by integrity checker

                # Convert all data to string for analysis
                all_text = ""
                non_empty_cells = 0
                total_cells = df.shape[0] * df.shape[1]

                for col in df.columns:
                    for value in df[col]:
                        if pd.notna(value) and str(value).strip():
                            all_text += str(value) + " "
                            non_empty_cells += 1

                # Check 1: Hebrew content
                hebrew_matches = hebrew_pattern.findall(all_text)
                has_hebrew = len(hebrew_matches) > 0

                if not has_hebrew:
                    issues["no_hebrew_content"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "sample_content": all_text[:100].strip(),
                        "table_name": table_name
                    })
                    file_valid = False
                else:
                    hebrew_stats["files_with_hebrew"] += 1
                    hebrew_stats["total_hebrew_chars"] += len(hebrew_matches)

                # Check 2: Empty table (all cells empty or whitespace)
                if non_empty_cells == 0:
                    issues["all_empty_cells"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "total_cells": total_cells
                    })
                    file_valid = False

                # Check 3: Table structure (reasonable content distribution)
                rows, cols = df.shape
                content_density = non_empty_cells / total_cells if total_cells > 0 else 0

                if content_density < 0.1:  # Less than 10% of cells have content
                    issues["no_table_structure"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "rows": rows,
                        "cols": cols,
                        "content_density": f"{content_density:.2%}",
                        "non_empty_cells": non_empty_cells
                    })
                    file_valid = False

                # Check 4: Suspicious content patterns
                suspicious_patterns = []

                # Check for tables that are mostly numbers (might be extracted incorrectly)
                digit_chars = len(re.findall(r'\d', all_text))
                total_chars = len(all_text.replace(' ', ''))

                if total_chars > 0 and (digit_chars / total_chars) > 0.8:
                    suspicious_patterns.append("mostly_numeric")

                # Check for repeated identical content (possible extraction duplication)
                words = all_text.split()
                if len(words) > 10:
                    unique_words = set(words)
                    if len(unique_words) / len(words) < 0.3:  # Less than 30% unique words
                        suspicious_patterns.append("repetitive_content")

                # Check for extremely long single cells (possible formatting issue)
                max_cell_length = 0
                for col in df.columns:
                    for value in df[col]:
                        if pd.notna(value):
                            cell_length = len(str(value))
                            max_cell_length = max(max_cell_length, cell_length)

                if max_cell_length > 500:  # Single cell with >500 characters
                    suspicious_patterns.append("oversized_cells")

                if suspicious_patterns:
                    issues["suspicious_content"].append({
                        "identifier": identifier,
                        "path": csv_path,
                        "patterns": suspicious_patterns,
                        "details": {
                            "digit_ratio": f"{digit_chars/total_chars:.2%}" if total_chars > 0 else "N/A",
                            "max_cell_length": max_cell_length,
                            "content_density": f"{content_density:.2%}"
                        }
                    })

                if file_valid:
                    valid_files += 1

            except Exception as e:
                issues["suspicious_content"].append({
                    "identifier": identifier,
                    "path": csv_path,
                    "patterns": ["read_error"],
                    "error": str(e)
                })

        except Exception as e:
            print(f"⚠️  Error processing {identifier}: {e}")

    # Generate report
    print("\n" + "="*60)
    print("🔤 CONTENT PATTERN VALIDATION REPORT")
    print("="*60)

    total_issues = sum(len(issue_list) for issue_list in issues.values())

    print(f"📊 Files processed: {processed}")
    print(f"✅ Valid patterns: {valid_files}")
    print(f"❌ Pattern issues: {total_issues}")
    print(f"🔤 Files with Hebrew: {hebrew_stats['files_with_hebrew']} ({hebrew_stats['files_with_hebrew']/processed:.1%})")
    print(f"🔤 Total Hebrew characters: {hebrew_stats['total_hebrew_chars']:,}")

    for issue_type, issue_list in issues.items():
        if issue_list:
            print(f"\n🚨 {issue_type.replace('_', ' ').upper()} ({len(issue_list)}):")
            for item in issue_list[:3]:  # Show first 3
                if issue_type == "no_hebrew_content":
                    print(f"  • {item['identifier']}")
                    print(f"    Sample: '{item['sample_content']}'")
                elif issue_type == "all_empty_cells":
                    print(f"  • {item['identifier']}: {item['total_cells']} total cells, all empty")
                elif issue_type == "no_table_structure":
                    print(f"  • {item['identifier']}: {item['rows']}x{item['cols']}, density: {item['content_density']}")
                elif issue_type == "suspicious_content":
                    print(f"  • {item['identifier']}: {', '.join(item['patterns'])}")

            if len(issue_list) > 3:
                print(f"  ... and {len(issue_list) - 3} more")

    # Calculate pattern validity score
    pattern_score = (valid_files / processed) * 100 if processed > 0 else 0

    print(f"\n📈 PATTERN VALIDITY SCORE: {pattern_score:.1f}% ({valid_files}/{processed})")

    # Save detailed report
    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_content_patterns.json")
    report = {
        "summary": {
            "files_processed": processed,
            "valid_patterns": valid_files,
            "total_issues": total_issues,
            "pattern_score": pattern_score,
            "hebrew_stats": hebrew_stats
        },
        "issues": issues
    }

    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed report saved to: {report_path}")

    return pattern_score > 85  # Consider success if >85% files have valid patterns

if __name__ == "__main__":
    # Run pattern validation
    is_valid = validate_content_patterns()

    if is_valid:
        print("\n✅ Content pattern validation PASSED")
    else:
        print("\n❌ Content pattern validation FAILED - review pattern issues")

🔤 Validating content patterns for 6732 CSV files...

🔤 CONTENT PATTERN VALIDATION REPORT
📊 Files processed: 6732
✅ Valid patterns: 6732
❌ Pattern issues: 1906
🔤 Files with Hebrew: 6732 (100.0%)
🔤 Total Hebrew characters: 8,807,636

🚨 SUSPICIOUS CONTENT (1906):
  • 2_06_2001: repetitive_content
  • 2_03_2001: repetitive_content
  • 14_03_2001: repetitive_content
  ... and 1903 more

📈 PATTERN VALIDITY SCORE: 100.0% (6732/6732)

💾 Detailed report saved to: /content/tables/validations/validation_content_patterns.json

✅ Content pattern validation PASSED


In [ ]:
import os
import json
from docx import Document
from pathlib import Path

def map_table_positions(base_dir="/content/reports", tables_dir="/content/tables"):
    """
    Re-scan original documents to map positions of all "לוח" tables
    and compare against extracted tables to detect gaps.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load extraction summary
    with open(summary_path, "r", encoding="utf-8") as f:
        extracted_summary = json.load(f)

    print(f"📍 Mapping table positions and comparing against {len(extracted_summary)} extracted tables...")

    position_map = {}
    comparison_results = {
        "documents_processed": 0,
        "documents_with_gaps": [],
        "position_mismatches": [],
        "extraction_summary": {}
    }

    # Process each year and document
    for year in range(2001, 2025):
        year_path = os.path.join(base_dir, str(year))
        if not os.path.isdir(year_path):
            continue

        for fname in os.listdir(year_path):
            if not fname.endswith(".docx"):
                continue

            chapter = fname.split("_")[0]
            fpath = os.path.join(year_path, fname)
            doc_key = f"{chapter}_{year}"

            try:
                doc = Document(fpath)
                comparison_results["documents_processed"] += 1

                # Find all "לוח" tables with their positions
                found_tables = []

                for table_idx, table in enumerate(doc.tables):
                    # Check if this table contains "לוח"
                    has_luach = False
                    luach_text = ""

                    if len(table.rows) > 0:
                        for cell in table.rows[0].cells:
                            if "לוח" in cell.text:
                                has_luach = True
                                luach_text = cell.text.strip()
                                break

                    if has_luach:
                        found_tables.append({
                            "document_table_index": table_idx,
                            "serial_in_doc": len(found_tables) + 1,
                            "luach_text": luach_text,
                            "table_dimensions": f"{len(table.rows)}x{len(table.columns) if table.rows else 0}"
                        })

                # Store position mapping
                position_map[doc_key] = {
                    "document_path": fpath,
                    "found_tables": found_tables,
                    "total_luach_tables": len(found_tables)
                }

                # Compare against extracted tables
                extracted_for_doc = {}
                for identifier, table_name in extracted_summary.items():
                    parts = identifier.split("_")
                    if len(parts) == 3:
                        serial, id_chapter, id_year = parts
                        if f"{id_chapter}_{id_year}" == doc_key:
                            extracted_for_doc[int(serial)] = {
                                "identifier": identifier,
                                "table_name": table_name
                            }

                # Analyze gaps and mismatches
                found_count = len(found_tables)
                extracted_count = len(extracted_for_doc)
                extracted_serials = sorted(extracted_for_doc.keys())

                doc_analysis = {
                    "found_in_document": found_count,
                    "extracted_count": extracted_count,
                    "extracted_serials": extracted_serials,
                    "expected_serials": list(range(1, found_count + 1)) if found_count > 0 else []
                }

                # Check for missing serials (gaps)
                if found_count > 0:
                    expected_serials = set(range(1, found_count + 1))
                    actual_serials = set(extracted_serials)
                    missing_serials = expected_serials - actual_serials
                    extra_serials = actual_serials - expected_serials

                    if missing_serials or extra_serials or found_count != extracted_count:
                        comparison_results["documents_with_gaps"].append({
                            "document": doc_key,
                            "path": fpath,
                            "found_tables": found_count,
                            "extracted_tables": extracted_count,
                            "missing_serials": sorted(list(missing_serials)),
                            "extra_serials": sorted(list(extra_serials)),
                            "gap_analysis": {
                                "missing_count": len(missing_serials),
                                "extra_count": len(extra_serials),
                                "extraction_rate": f"{extracted_count}/{found_count}" if found_count > 0 else "0/0"
                            }
                        })

                comparison_results["extraction_summary"][doc_key] = doc_analysis

            except Exception as e:
                print(f"⚠️  Error processing {fpath}: {e}")
                comparison_results["position_mismatches"].append({
                    "document": doc_key,
                    "path": fpath,
                    "error": str(e)
                })

    # Generate report
    print("\n" + "="*60)
    print("📍 TABLE POSITION MAPPING REPORT")
    print("="*60)

    total_docs = comparison_results["documents_processed"]
    docs_with_gaps = len(comparison_results["documents_with_gaps"])
    docs_perfect = total_docs - docs_with_gaps - len(comparison_results["position_mismatches"])

    print(f"📄 Documents processed: {total_docs}")
    print(f"✅ Perfect extractions: {docs_perfect}")
    print(f"⚠️  Documents with gaps: {docs_with_gaps}")
    print(f"❌ Processing errors: {len(comparison_results['position_mismatches'])}")

    if comparison_results["documents_with_gaps"]:
        print(f"\n🔍 EXTRACTION GAPS DETECTED ({docs_with_gaps} documents):")
        for gap_info in comparison_results["documents_with_gaps"][:5]:  # Show first 5
            gap = gap_info["gap_analysis"]
            print(f"  • {gap_info['document']}: {gap['extraction_rate']}")
            if gap_info["missing_serials"]:
                print(f"    Missing serials: {gap_info['missing_serials']}")
            if gap_info["extra_serials"]:
                print(f"    Extra serials: {gap_info['extra_serials']}")

        if docs_with_gaps > 5:
            print(f"  ... and {docs_with_gaps - 5} more documents with gaps")

    if comparison_results["position_mismatches"]:
        print(f"\n❌ PROCESSING ERRORS ({len(comparison_results['position_mismatches'])}):")
        for error in comparison_results["position_mismatches"][:3]:
            print(f"  • {error['document']}: {error['error']}")

    # Calculate extraction completeness
    total_found = sum(doc_info.get("total_luach_tables", 0) for doc_info in position_map.values())
    total_extracted = len(extracted_summary)
    completeness_rate = (total_extracted / total_found) * 100 if total_found > 0 else 0

    print(f"\n📊 OVERALL EXTRACTION COMPLETENESS:")
    print(f"   Tables found in documents: {total_found}")
    print(f"   Tables extracted: {total_extracted}")
    print(f"   Completeness rate: {completeness_rate:.1f}%")

    # Save detailed mapping
    mapping_report = {
        "summary": {
            "documents_processed": total_docs,
            "perfect_extractions": docs_perfect,
            "documents_with_gaps": docs_with_gaps,
            "total_tables_found": total_found,
            "total_tables_extracted": total_extracted,
            "completeness_rate": completeness_rate
        },
        "position_mapping": position_map,
        "gap_analysis": comparison_results
    }

    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_position_mapping.json")
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(mapping_report, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed mapping saved to: {report_path}")

    return completeness_rate > 95  # Consider success if >95% completeness

if __name__ == "__main__":
    # Run position mapping
    is_complete = map_table_positions()

    if is_complete:
        print("\n✅ Table position mapping validation PASSED")
    else:
        print("\n❌ Table position mapping validation FAILED - review extraction gaps")

📍 Mapping table positions and comparing against 6732 extracted tables...

📍 TABLE POSITION MAPPING REPORT
📄 Documents processed: 346
✅ Perfect extractions: 328
⚠️  Documents with gaps: 18
❌ Processing errors: 0

🔍 EXTRACTION GAPS DETECTED (18 documents):
  • 07_2001: 32/33
    Missing serials: [33]
  • 07_2002: 27/28
    Missing serials: [28]
  • 07_2003: 25/26
    Missing serials: [26]
  • 14_2007: 37/38
    Missing serials: [38]
  • 14_2010: 39/40
    Missing serials: [40]
  ... and 13 more documents with gaps

📊 OVERALL EXTRACTION COMPLETENESS:
   Tables found in documents: 6750
   Tables extracted: 6732
   Completeness rate: 99.7%

💾 Detailed mapping saved to: /content/tables/validations/validation_position_mapping.json

✅ Table position mapping validation PASSED


In [ ]:
import os
import json
from pathlib import Path

def validate_file_completeness(tables_dir="/content/tables"):
    """
    Validate that all CSV files referenced in tables_summary.json exist
    and follow the expected naming convention.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    # Check if summary file exists
    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load summary data
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"📊 Checking {len(summary)} table entries...")

    missing_files = []
    naming_issues = []
    found_files = []

    for identifier, table_name in summary.items():
        # Parse identifier: serial_chapter_year
        try:
            parts = identifier.split("_")
            if len(parts) != 3:
                naming_issues.append(f"Invalid identifier format: {identifier}")
                continue

            serial, chapter, year = parts

            # Construct expected file path
            expected_path = os.path.join(tables_dir, year, chapter, f"{identifier}.csv")

            if os.path.exists(expected_path):
                found_files.append(expected_path)
            else:
                missing_files.append({
                    "identifier": identifier,
                    "expected_path": expected_path,
                    "table_name": table_name
                })

        except Exception as e:
            naming_issues.append(f"Error parsing identifier {identifier}: {e}")

    # Generate report
    print("\n" + "="*60)
    print("📋 FILE COMPLETENESS VALIDATION REPORT")
    print("="*60)

    print(f"✅ Files found: {len(found_files)}")
    print(f"❌ Files missing: {len(missing_files)}")
    print(f"⚠️  Naming issues: {len(naming_issues)}")

    if missing_files:
        print(f"\n🔍 MISSING FILES ({len(missing_files)}):")
        for item in missing_files[:10]:  # Show first 10
            print(f"  • {item['identifier']} → {item['expected_path']}")
            print(f"    Table: {item['table_name'][:50]}...")
        if len(missing_files) > 10:
            print(f"  ... and {len(missing_files) - 10} more")

    if naming_issues:
        print(f"\n⚠️  NAMING ISSUES ({len(naming_issues)}):")
        for issue in naming_issues[:5]:  # Show first 5
            print(f"  • {issue}")
        if len(naming_issues) > 5:
            print(f"  ... and {len(naming_issues) - 5} more")

    # Calculate success rate
    total_expected = len(summary)
    success_rate = (len(found_files) / total_expected) * 100 if total_expected > 0 else 0

    print(f"\n📈 SUCCESS RATE: {success_rate:.1f}% ({len(found_files)}/{total_expected})")

    # Save detailed report
    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_file_completeness.json")
    report = {
        "summary": {
            "total_expected": total_expected,
            "files_found": len(found_files),
            "files_missing": len(missing_files),
            "naming_issues": len(naming_issues),
            "success_rate": success_rate
        },
        "missing_files": missing_files,
        "naming_issues": naming_issues,
        "found_files": found_files
    }

    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed report saved to: {report_path}")

    return success_rate > 95  # Consider success if >95% files found

if __name__ == "__main__":
    # Run validation
    is_valid = validate_file_completeness()

    if is_valid:
        print("\n✅ File completeness validation PASSED")
    else:
        print("\n❌ File completeness validation FAILED - review missing files")

📊 Checking 6732 table entries...

📋 FILE COMPLETENESS VALIDATION REPORT
✅ Files found: 6732
❌ Files missing: 0
⚠️  Naming issues: 0

📈 SUCCESS RATE: 100.0% (6732/6732)

💾 Detailed report saved to: /content/tables/validations/validation_file_completeness.json

✅ File completeness validation PASSED


In [ ]:
import os
import json
from collections import defaultdict

def validate_serial_sequences(tables_dir="/content/tables"):
    """
    Validate that serial numbers within each document are consecutive
    (1, 2, 3... with no gaps) by analyzing the tables_summary.json.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load extraction summary
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"🔢 Validating serial number sequences for {len(summary)} extracted tables...")

    # Group by document (chapter_year)
    documents = defaultdict(list)

    for identifier, table_name in summary.items():
        try:
            parts = identifier.split("_")
            if len(parts) != 3:
                print(f"⚠️  Invalid identifier format: {identifier}")
                continue

            serial, chapter, year = parts
            doc_key = f"{chapter}_{year}"

            # Validate serial is numeric
            try:
                serial_num = int(serial)
            except ValueError:
                print(f"⚠️  Non-numeric serial in {identifier}: {serial}")
                continue

            documents[doc_key].append({
                "serial": serial_num,
                "identifier": identifier,
                "table_name": table_name
            })

        except Exception as e:
            print(f"⚠️  Error parsing identifier {identifier}: {e}")

    # Analyze sequences for each document
    sequence_issues = {
        "missing_serials": [],
        "duplicate_serials": [],
        "non_consecutive": [],
        "starting_issues": []
    }

    valid_documents = 0
    total_documents = len(documents)

    for doc_key, tables in documents.items():
        # Sort by serial number
        tables.sort(key=lambda x: x["serial"])
        serials = [t["serial"] for t in tables]

        doc_issues = []

        # Check 1: Should start with 1
        if serials and serials[0] != 1:
            sequence_issues["starting_issues"].append({
                "document": doc_key,
                "first_serial": serials[0],
                "expected": 1,
                "serials": serials
            })
            doc_issues.append("wrong_start")

        # Check 2: Look for duplicates
        seen_serials = set()
        duplicates = []
        for table in tables:
            serial = table["serial"]
            if serial in seen_serials:
                duplicates.append(serial)
            seen_serials.add(serial)

        if duplicates:
            sequence_issues["duplicate_serials"].append({
                "document": doc_key,
                "duplicates": list(set(duplicates)),
                "serials": serials,
                "affected_tables": [t["identifier"] for t in tables if t["serial"] in duplicates]
            })
            doc_issues.append("duplicates")

        # Check 3: Look for missing serials (gaps in sequence)
        if serials:
            expected_max = max(serials)
            expected_sequence = set(range(1, expected_max + 1))
            actual_sequence = set(serials)
            missing = expected_sequence - actual_sequence

            if missing:
                sequence_issues["missing_serials"].append({
                    "document": doc_key,
                    "missing_serials": sorted(list(missing)),
                    "present_serials": serials,
                    "expected_range": f"1-{expected_max}",
                    "gap_count": len(missing)
                })
                doc_issues.append("missing")

        # Check 4: Non-consecutive (should be 1,2,3,4... not 1,3,5,7...)
        if serials and not duplicates:  # Only if no duplicates
            for i in range(len(serials) - 1):
                if serials[i+1] != serials[i] + 1:
                    sequence_issues["non_consecutive"].append({
                        "document": doc_key,
                        "serials": serials,
                        "break_points": [(serials[i], serials[i+1]) for i in range(len(serials)-1)
                                       if serials[i+1] != serials[i] + 1]
                    })
                    doc_issues.append("non_consecutive")
                    break

        # Count valid documents (no issues)
        if not doc_issues:
            valid_documents += 1

    # Generate report
    print("\n" + "="*60)
    print("🔢 SERIAL NUMBER SEQUENCE VALIDATION REPORT")
    print("="*60)

    total_issues = sum(len(issue_list) for issue_list in sequence_issues.values())

    print(f"📄 Documents analyzed: {total_documents}")
    print(f"✅ Perfect sequences: {valid_documents}")
    print(f"⚠️  Documents with issues: {total_documents - valid_documents}")
    print(f"🚨 Total sequence issues: {total_issues}")

    for issue_type, issue_list in sequence_issues.items():
        if issue_list:
            print(f"\n🔍 {issue_type.replace('_', ' ').upper()} ({len(issue_list)} documents):")

            for item in issue_list[:3]:  # Show first 3
                if issue_type == "missing_serials":
                    print(f"  • {item['document']}: Missing {item['missing_serials']} from range {item['expected_range']}")
                    print(f"    Present: {item['present_serials']}")
                elif issue_type == "duplicate_serials":
                    print(f"  • {item['document']}: Duplicate serials {item['duplicates']}")
                    print(f"    Affected: {', '.join(item['affected_tables'][:3])}")
                elif issue_type == "starting_issues":
                    print(f"  • {item['document']}: Starts with {item['first_serial']} instead of 1")
                    print(f"    Sequence: {item['serials']}")
                elif issue_type == "non_consecutive":
                    print(f"  • {item['document']}: Non-consecutive sequence")
                    print(f"    Breaks: {item['break_points']}")

            if len(issue_list) > 3:
                print(f"  ... and {len(issue_list) - 3} more")

    # Calculate sequence validity
    sequence_score = (valid_documents / total_documents) * 100 if total_documents > 0 else 0

    print(f"\n📈 SEQUENCE VALIDITY SCORE: {sequence_score:.1f}% ({valid_documents}/{total_documents})")

    # Generate document-level summary
    doc_summary = {}
    for doc_key, tables in documents.items():
        serials = sorted([t["serial"] for t in tables])
        doc_summary[doc_key] = {
            "table_count": len(tables),
            "serial_range": f"{min(serials)}-{max(serials)}" if serials else "empty",
            "is_consecutive": len(serials) > 0 and serials == list(range(serials[0], serials[-1] + 1)),
            "starts_with_one": len(serials) > 0 and serials[0] == 1,
            "has_duplicates": len(serials) != len(set(serials))
        }

    # Save detailed report
    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_serial_sequences.json")
    report = {
        "summary": {
            "documents_analyzed": total_documents,
            "perfect_sequences": valid_documents,
            "documents_with_issues": total_documents - valid_documents,
            "total_issues": total_issues,
            "sequence_score": sequence_score
        },
        "issues": sequence_issues,
        "document_summary": doc_summary
    }

    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(report, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed report saved to: {report_path}")

    return sequence_score > 90  # Consider success if >90% documents have perfect sequences

if __name__ == "__main__":
    # Run sequence validation
    is_valid = validate_serial_sequences()

    if is_valid:
        print("\n✅ Serial number sequence validation PASSED")
    else:
        print("\n❌ Serial number sequence validation FAILED - review sequence issues")

🔢 Validating serial number sequences for 6732 extracted tables...

🔢 SERIAL NUMBER SEQUENCE VALIDATION REPORT
📄 Documents analyzed: 316
✅ Perfect sequences: 316
⚠️  Documents with issues: 0
🚨 Total sequence issues: 0

📈 SEQUENCE VALIDITY SCORE: 100.0% (316/316)

💾 Detailed report saved to: /content/tables/validations/validation_serial_sequences.json

✅ Serial number sequence validation PASSED


In [ ]:
import os
import json
import statistics
from collections import defaultdict

def analyze_cross_chapter_patterns(tables_dir="/content/tables"):
    """
    Track historical patterns for each chapter name and flag extreme deviations
    (>50% change from historical range) to identify potential extraction issues.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load extraction summary
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"📊 Analyzing cross-chapter patterns for {len(summary)} extracted tables...")

    # Group tables by chapter and year
    chapter_data = defaultdict(lambda: defaultdict(int))  # chapter -> year -> count

    for identifier, table_name in summary.items():
        try:
            parts = identifier.split("_")
            if len(parts) != 3:
                continue

            serial, chapter, year = parts
            chapter_data[chapter][int(year)] += 1

        except Exception as e:
            print(f"⚠️  Error parsing identifier {identifier}: {e}")

    # Analyze patterns for each chapter
    analysis_results = {
        "chapter_patterns": {},
        "flagged_chapters": [],
        "year_comparisons": [],
        "statistical_summary": {}
    }

    flagged_count = 0
    total_chapters = len(chapter_data)

    for chapter, year_counts in chapter_data.items():
        years = sorted(year_counts.keys())
        counts = [year_counts[year] for year in years]

        if len(counts) < 2:  # Need at least 2 years for comparison
            continue

        # Calculate statistical measures
        mean_count = statistics.mean(counts)
        median_count = statistics.median(counts)

        if len(counts) >= 3:
            std_dev = statistics.stdev(counts)
        else:
            std_dev = 0

        min_count = min(counts)
        max_count = max(counts)

        # Calculate historical range (mean ± 1 std dev, but at least min-max range)
        if std_dev > 0:
            lower_bound = max(min_count, mean_count - std_dev)
            upper_bound = min(max_count, mean_count + std_dev)
        else:
            lower_bound = min_count
            upper_bound = max_count

        # Define extreme deviation threshold (50% outside historical range)
        deviation_threshold = 0.5
        extreme_lower = lower_bound * (1 - deviation_threshold)
        extreme_upper = upper_bound * (1 + deviation_threshold)

        # Find outliers (years with extreme deviations)
        outliers = []
        for year in years:
            count = year_counts[year]
            if count < extreme_lower or count > extreme_upper:
                deviation_pct = abs(count - mean_count) / mean_count * 100 if mean_count > 0 else 0
                outliers.append({
                    "year": year,
                    "count": count,
                    "expected_range": f"{lower_bound:.1f}-{upper_bound:.1f}",
                    "deviation_percent": deviation_pct,
                    "type": "low" if count < extreme_lower else "high"
                })

        # Store chapter pattern analysis
        chapter_pattern = {
            "years_analyzed": years,
            "table_counts": counts,
            "statistics": {
                "mean": round(mean_count, 1),
                "median": median_count,
                "std_dev": round(std_dev, 1),
                "min": min_count,
                "max": max_count,
                "range": f"{min_count}-{max_count}"
            },
            "historical_bounds": {
                "lower": round(lower_bound, 1),
                "upper": round(upper_bound, 1)
            },
            "extreme_thresholds": {
                "lower": round(extreme_lower, 1),
                "upper": round(extreme_upper, 1)
            },
            "outliers": outliers,
            "stability_score": round((1 - (std_dev / mean_count)) * 100, 1) if mean_count > 0 else 0
        }

        analysis_results["chapter_patterns"][chapter] = chapter_pattern

        # Flag chapters with outliers
        if outliers:
            flagged_count += 1
            analysis_results["flagged_chapters"].append({
                "chapter": chapter,
                "outlier_count": len(outliers),
                "outliers": outliers,
                "severity": "high" if any(o["deviation_percent"] > 75 for o in outliers) else "medium"
            })

    # Generate year-over-year comparison
    all_years = sorted(set(year for chapter_years in chapter_data.values() for year in chapter_years.keys()))

    for i in range(len(all_years) - 1):
        year1, year2 = all_years[i], all_years[i + 1]

        year_comparison = {
            "years": f"{year1}-{year2}",
            "chapter_changes": []
        }

        for chapter in chapter_data:
            count1 = chapter_data[chapter].get(year1, 0)
            count2 = chapter_data[chapter].get(year2, 0)

            if count1 > 0 and count2 > 0:  # Both years have data
                change_pct = ((count2 - count1) / count1) * 100
                if abs(change_pct) > 50:  # Significant change
                    year_comparison["chapter_changes"].append({
                        "chapter": chapter,
                        "from_count": count1,
                        "to_count": count2,
                        "change_percent": round(change_pct, 1)
                    })

        if year_comparison["chapter_changes"]:
            analysis_results["year_comparisons"].append(year_comparison)

    # Overall statistical summary
    all_counts = []
    for chapter_pattern in analysis_results["chapter_patterns"].values():
        all_counts.extend(chapter_pattern["table_counts"])

    if all_counts:
        analysis_results["statistical_summary"] = {
            "total_data_points": len(all_counts),
            "overall_mean": round(statistics.mean(all_counts), 1),
            "overall_median": statistics.median(all_counts),
            "overall_std": round(statistics.stdev(all_counts), 1) if len(all_counts) > 1 else 0,
            "chapters_analyzed": total_chapters,
            "chapters_flagged": flagged_count,
            "stability_rate": round((1 - flagged_count / total_chapters) * 100, 1) if total_chapters > 0 else 0
        }

    # Generate report
    print("\n" + "="*60)
    print("📊 CROSS-CHAPTER PATTERN ANALYSIS REPORT")
    print("="*60)

    summary_stats = analysis_results["statistical_summary"]
    print(f"📈 Chapters analyzed: {summary_stats['chapters_analyzed']}")
    print(f"⚠️  Chapters flagged: {summary_stats['chapters_flagged']}")
    print(f"📊 Stability rate: {summary_stats['stability_rate']}%")
    print(f"📋 Overall mean tables/chapter: {summary_stats['overall_mean']}")
    print(f"📋 Overall std deviation: {summary_stats['overall_std']}")

    if analysis_results["flagged_chapters"]:
        print(f"\n🚨 FLAGGED CHAPTERS FOR MANUAL VERIFICATION ({len(analysis_results['flagged_chapters'])}):")

        # Sort by severity and deviation
        flagged_sorted = sorted(analysis_results["flagged_chapters"],
                              key=lambda x: (x["severity"] == "high", x["outlier_count"]),
                              reverse=True)

        for flag_info in flagged_sorted[:5]:  # Show top 5
            print(f"  • {flag_info['chapter']} ({flag_info['severity']} severity)")
            for outlier in flag_info["outliers"][:2]:  # Show first 2 outliers
                print(f"    - {outlier['year']}: {outlier['count']} tables " +
                      f"({outlier['deviation_percent']:.1f}% deviation, expected {outlier['expected_range']})")

        if len(analysis_results["flagged_chapters"]) > 5:
            print(f"  ... and {len(analysis_results['flagged_chapters']) - 5} more")

    if analysis_results["year_comparisons"]:
        print(f"\n📅 SIGNIFICANT YEAR-OVER-YEAR CHANGES ({len(analysis_results['year_comparisons'])} periods):")
        for comparison in analysis_results["year_comparisons"][:3]:  # Show first 3
            print(f"  • {comparison['years']}: {len(comparison['chapter_changes'])} chapters with >50% change")

    # Save detailed report
    report_path = os.path.join(tables_dir, "validation_cross_chapter_analysis.json")
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed analysis saved to: {report_path}")

    # Generate manual verification priority list
    verification_list = []
    for flag_info in flagged_sorted:
        for outlier in flag_info["outliers"]:
            verification_list.append({
                "priority": "high" if flag_info["severity"] == "high" else "medium",
                "chapter": flag_info["chapter"],
                "year": outlier["year"],
                "extracted_count": outlier["count"],
                "expected_range": outlier["expected_range"],
                "deviation": f"{outlier['deviation_percent']:.1f}%",
                "verification_action": "Check TOC manually"
            })

    # Save verification priority list
    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    verification_path = os.path.join(tables_dir, "validations", "manual_verification_priority.json")
    with open(verification_path, "w", encoding="utf-8") as f:
        json.dump(verification_list, f, ensure_ascii=False, indent=2)

    print(f"📋 Manual verification list saved to: {verification_path}")
    print(f"   → {len(verification_list)} chapter-year combinations need TOC verification")

    return summary_stats["stability_rate"] > 75  # Consider success if >75% chapters are stable

if __name__ == "__main__":
    # Run cross-chapter analysis
    is_stable = analyze_cross_chapter_patterns()

    if is_stable:
        print("\n✅ Cross-chapter pattern analysis PASSED")
    else:
        print("\n❌ Cross-chapter pattern analysis FAILED - many chapters flagged for review")

📊 Analyzing cross-chapter patterns for 6732 extracted tables...

📊 CROSS-CHAPTER PATTERN ANALYSIS REPORT
📈 Chapters analyzed: 15
⚠️  Chapters flagged: 13
📊 Stability rate: 13.3%
📋 Overall mean tables/chapter: 21.3
📋 Overall std deviation: 14.1

🚨 FLAGGED CHAPTERS FOR MANUAL VERIFICATION (13):
  • 01 (high severity)
    - 2012: 31 tables (137.6% deviation, expected 5.9-20.2)
    - 2017: 2 tables (84.7% deviation, expected 5.9-20.2)
  • 12 (high severity)
    - 2017: 1 tables (97.0% deviation, expected 16.3-50.1)
    - 2018: 1 tables (97.0% deviation, expected 16.3-50.1)
  • 14 (high severity)
    - 2017: 1 tables (96.8% deviation, expected 15.0-48.0)
    - 2018: 2 tables (93.7% deviation, expected 15.0-48.0)
  • 03 (high severity)
    - 2017: 1 tables (94.2% deviation, expected 10.2-24.4)
    - 2018: 1 tables (94.2% deviation, expected 10.2-24.4)
  • 13 (high severity)
    - 2017: 1 tables (95.9% deviation, expected 13.1-35.8)
    - 2018: 1 tables (95.9% deviation, expected 13.1-35.8)
 

In [ ]:
import os
import json
import pandas as pd
import statistics
from collections import defaultdict

def analyze_table_size_distribution(tables_dir="/content/tables"):
    """
    Analyze table size distributions to detect extraction errors through
    unusually small (1-2 rows) or extremely large tables.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load extraction summary
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"📏 Analyzing table size distributions for {len(summary)} tables...")

    table_sizes = []
    size_issues = {
        "too_small": [],
        "extremely_large": [],
        "unreadable": [],
        "suspicious_dimensions": []
    }

    processed = 0

    for identifier, table_name in summary.items():
        try:
            # Parse identifier and construct path
            parts = identifier.split("_")
            if len(parts) != 3:
                continue

            serial, chapter, year = parts
            csv_path = os.path.join(tables_dir, year, chapter, f"{identifier}.csv")

            if not os.path.exists(csv_path):
                continue  # Already handled by other validators

            processed += 1

            try:
                # Read CSV to get dimensions
                df = pd.read_csv(csv_path, encoding='utf-8-sig')
                rows, cols = df.shape

                # Calculate file size for additional context
                file_size_bytes = os.path.getsize(csv_path)
                file_size_kb = file_size_bytes / 1024

                table_info = {
                    "identifier": identifier,
                    "path": csv_path,
                    "rows": rows,
                    "cols": cols,
                    "file_size_kb": round(file_size_kb, 2),
                    "table_name": table_name,
                    "chapter": chapter,
                    "year": int(year)
                }

                table_sizes.append(table_info)

                # Flag 1: Too small (suspicious)
                if rows <= 2:  # Header + 1 row or less
                    size_issues["too_small"].append({
                        **table_info,
                        "issue": f"Only {rows} rows (likely header only or truncated)"
                    })

                # Flag 2: Extremely large (possible extraction error)
                if rows > 500 or cols > 30:
                    size_issues["extremely_large"].append({
                        **table_info,
                        "issue": f"Unusually large: {rows}x{cols} (possible merged tables or extraction error)"
                    })

                # Flag 3: Suspicious dimensions
                suspicious_reasons = []

                # Single column tables (unusual for Hebrew statistical tables)
                if cols == 1 and rows > 2:
                    suspicious_reasons.append("single_column")

                # Very wide tables (might be formatting issues)
                if cols > 20:
                    suspicious_reasons.append("too_wide")

                # Extremely rectangular (might indicate extraction problems)
                if rows > 0 and cols > 0:
                    aspect_ratio = max(rows, cols) / min(rows, cols)
                    if aspect_ratio > 20:  # One dimension is 20x larger than other
                        suspicious_reasons.append("extreme_aspect_ratio")

                # Large file size but small table (might indicate extraction errors)
                if file_size_kb > 100 and rows * cols < 50:
                    suspicious_reasons.append("large_file_small_table")

                if suspicious_reasons:
                    size_issues["suspicious_dimensions"].append({
                        **table_info,
                        "suspicious_patterns": suspicious_reasons,
                        "aspect_ratio": round(max(rows, cols) / min(rows, cols), 1) if rows > 0 and cols > 0 else 0
                    })

            except Exception as e:
                size_issues["unreadable"].append({
                    "identifier": identifier,
                    "path": csv_path,
                    "error": str(e),
                    "table_name": table_name
                })

        except Exception as e:
            print(f"⚠️  Error processing {identifier}: {e}")

    # Statistical analysis of table sizes
    if table_sizes:
        rows_data = [t["rows"] for t in table_sizes]
        cols_data = [t["cols"] for t in table_sizes]
        size_data = [t["file_size_kb"] for t in table_sizes]

        # Calculate statistics
        stats = {
            "rows": {
                "mean": round(statistics.mean(rows_data), 1),
                "median": statistics.median(rows_data),
                "std": round(statistics.stdev(rows_data), 1) if len(rows_data) > 1 else 0,
                "min": min(rows_data),
                "max": max(rows_data),
                "quartiles": [
                    round(statistics.quantiles(rows_data, n=4)[0], 1) if len(rows_data) >= 4 else min(rows_data),
                    round(statistics.quantiles(rows_data, n=4)[2], 1) if len(rows_data) >= 4 else max(rows_data)
                ]
            },
            "cols": {
                "mean": round(statistics.mean(cols_data), 1),
                "median": statistics.median(cols_data),
                "std": round(statistics.stdev(cols_data), 1) if len(cols_data) > 1 else 0,
                "min": min(cols_data),
                "max": max(cols_data),
                "quartiles": [
                    round(statistics.quantiles(cols_data, n=4)[0], 1) if len(cols_data) >= 4 else min(cols_data),
                    round(statistics.quantiles(cols_data, n=4)[2], 1) if len(cols_data) >= 4 else max(cols_data)
                ]
            },
            "file_size_kb": {
                "mean": round(statistics.mean(size_data), 1),
                "median": round(statistics.median(size_data), 1),
                "std": round(statistics.stdev(size_data), 1) if len(size_data) > 1 else 0,
                "min": round(min(size_data), 2),
                "max": round(max(size_data), 2)
            }
        }

        # Identify statistical outliers using IQR method
        def find_outliers(data, multiplier=1.5):
            if len(data) < 4:
                return []
            q1, q3 = statistics.quantiles(data, n=4)[0], statistics.quantiles(data, n=4)[2]
            iqr = q3 - q1
            lower_bound = q1 - multiplier * iqr
            upper_bound = q3 + multiplier * iqr
            return [x for x in data if x < lower_bound or x > upper_bound]

        row_outliers = find_outliers(rows_data)
        col_outliers = find_outliers(cols_data)

        # Group tables by year for trend analysis
        yearly_stats = defaultdict(lambda: {"count": 0, "avg_rows": 0, "avg_cols": 0})
        for table in table_sizes:
            year = table["year"]
            yearly_stats[year]["count"] += 1
            yearly_stats[year]["avg_rows"] += table["rows"]
            yearly_stats[year]["avg_cols"] += table["cols"]

        for year in yearly_stats:
            count = yearly_stats[year]["count"]
            yearly_stats[year]["avg_rows"] = round(yearly_stats[year]["avg_rows"] / count, 1)
            yearly_stats[year]["avg_cols"] = round(yearly_stats[year]["avg_cols"] / count, 1)

    # Generate report
    print("\n" + "="*60)
    print("📏 TABLE SIZE DISTRIBUTION ANALYSIS REPORT")
    print("="*60)

    total_issues = sum(len(issue_list) for issue_list in size_issues.values())

    print(f"📊 Tables analyzed: {processed}")
    print(f"⚠️  Size anomalies detected: {total_issues}")

    if table_sizes:
        print(f"\n📈 SIZE STATISTICS:")
        print(f"   Rows: μ={stats['rows']['mean']}, σ={stats['rows']['std']}, range={stats['rows']['min']}-{stats['rows']['max']}")
        print(f"   Cols: μ={stats['cols']['mean']}, σ={stats['cols']['std']}, range={stats['cols']['min']}-{stats['cols']['max']}")
        print(f"   File Size: μ={stats['file_size_kb']['mean']}KB, range={stats['file_size_kb']['min']}-{stats['file_size_kb']['max']}KB")

        if row_outliers or col_outliers:
            print(f"   Statistical outliers: {len(set(row_outliers + col_outliers))} tables")

    for issue_type, issue_list in size_issues.items():
        if issue_list:
            print(f"\n🚨 {issue_type.replace('_', ' ').upper()} ({len(issue_list)}):")

            for item in issue_list[:3]:  # Show first 3
                if issue_type == "too_small":
                    print(f"  • {item['identifier']}: {item['rows']}x{item['cols']} - {item['issue']}")
                elif issue_type == "extremely_large":
                    print(f"  • {item['identifier']}: {item['rows']}x{item['cols']} ({item['file_size_kb']}KB)")
                elif issue_type == "suspicious_dimensions":
                    patterns = ", ".join(item['suspicious_patterns'])
                    print(f"  • {item['identifier']}: {item['rows']}x{item['cols']} - {patterns}")
                elif issue_type == "unreadable":
                    print(f"  • {item['identifier']}: {item['error']}")

            if len(issue_list) > 3:
                print(f"  ... and {len(issue_list) - 3} more")

    # Calculate size validity score
    size_score = ((processed - total_issues) / processed) * 100 if processed > 0 else 0

    print(f"\n📈 SIZE VALIDITY SCORE: {size_score:.1f}% ({processed - total_issues}/{processed})")

    # Save detailed report
    analysis_results = {
        "summary": {
            "tables_analyzed": processed,
            "total_issues": total_issues,
            "size_validity_score": size_score,
            "statistics": stats if table_sizes else {}
        },
        "size_issues": size_issues,
        "yearly_trends": dict(yearly_stats) if table_sizes else {},
        "table_details": table_sizes
    }

    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_size_distribution.json")
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Detailed analysis saved to: {report_path}")

    return size_score > 85  # Consider success if >85% tables have normal sizes

if __name__ == "__main__":
    # Run size distribution analysis
    is_valid = analyze_table_size_distribution()

    if is_valid:
        print("\n✅ Table size distribution analysis PASSED")
    else:
        print("\n❌ Table size distribution analysis FAILED - many size anomalies detected")

📏 Analyzing table size distributions for 6732 tables...

📏 TABLE SIZE DISTRIBUTION ANALYSIS REPORT
📊 Tables analyzed: 6732
⚠️  Size anomalies detected: 469

📈 SIZE STATISTICS:
   Rows: μ=23.6, σ=26.2, range=1-209
   Cols: μ=8.0, σ=4.2, range=1-57
   File Size: μ=3.8KB, range=0.11-148.41KB
   Statistical outliers: 140 tables

🚨 TOO SMALL (260):
  • 1_07_2001: 1x1 - Only 1 rows (likely header only or truncated)
  • 8_14_2001: 1x1 - Only 1 rows (likely header only or truncated)
  • 1_07_2002: 1x1 - Only 1 rows (likely header only or truncated)
  ... and 257 more

🚨 EXTREMELY LARGE (24):
  • 12_10_2004: 193x35 (97.48KB)
  • 11_13_2004: 113x56 (82.53KB)
  • 9_10_2005: 194x35 (98.36KB)
  ... and 21 more

🚨 SUSPICIOUS DIMENSIONS (185):
  • 6_10_2001: 70x23 - too_wide
  • 4_03_2002: 178x23 - too_wide
  • 23_07_2002: 29x21 - too_wide
  ... and 182 more

📈 SIZE VALIDITY SCORE: 93.0% (6263/6732)

💾 Detailed analysis saved to: /content/tables/validations/validation_size_distribution.json

✅ Table 

In [ ]:
import os
import json
import pandas as pd
from collections import defaultdict

def generate_completeness_overview(tables_dir="/content/tables"):
    """
    Generate summary matrix showing table counts per chapter-year
    for visual pattern recognition and completeness assessment.
    """
    summary_path = os.path.join(tables_dir, "tables_summary.json")

    if not os.path.exists(summary_path):
        print(f"❌ ERROR: tables_summary.json not found at {summary_path}")
        return False

    # Load extraction summary
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)

    print(f"📋 Generating completeness overview for {len(summary)} extracted tables...")

    # Group data by chapter and year
    data_matrix = defaultdict(lambda: defaultdict(int))  # chapter -> year -> count
    all_chapters = set()
    all_years = set()

    for identifier, table_name in summary.items():
        try:
            parts = identifier.split("_")
            if len(parts) != 3:
                continue

            serial, chapter, year = parts
            year_int = int(year)

            data_matrix[chapter][year_int] += 1
            all_chapters.add(chapter)
            all_years.add(year_int)

        except Exception as e:
            print(f"⚠️  Error parsing identifier {identifier}: {e}")

    # Sort for consistent output
    all_chapters = sorted(all_chapters)
    all_years = sorted(all_years)

    # Create DataFrame for matrix representation
    matrix_data = []
    for chapter in all_chapters:
        row = {"Chapter": chapter}
        for year in all_years:
            row[str(year)] = data_matrix[chapter][year]
        matrix_data.append(row)

    df_matrix = pd.DataFrame(matrix_data)

    # Calculate summary statistics
    stats = {
        "total_chapters": len(all_chapters),
        "total_years": len(all_years),
        "year_range": f"{min(all_years)}-{max(all_years)}",
        "total_extractions": sum(sum(year_data.values()) for year_data in data_matrix.values())
    }

    # Analyze completeness patterns
    completeness_analysis = {
        "missing_data": [],  # Chapter-year combinations with 0 tables
        "low_extraction": [],  # Suspiciously low counts
        "high_extraction": [],  # Suspiciously high counts
        "chapter_statistics": {},
        "year_statistics": {}
    }

    # Analyze each chapter across years
    for chapter in all_chapters:
        chapter_counts = [data_matrix[chapter][year] for year in all_years]
        non_zero_counts = [c for c in chapter_counts if c > 0]

        chapter_stats = {
            "total_tables": sum(chapter_counts),
            "years_with_data": len(non_zero_counts),
            "years_missing": len(all_years) - len(non_zero_counts),
            "avg_tables_per_year": round(sum(non_zero_counts) / len(non_zero_counts), 1) if non_zero_counts else 0,
            "min_tables": min(non_zero_counts) if non_zero_counts else 0,
            "max_tables": max(non_zero_counts) if non_zero_counts else 0,
            "consistency_score": 0
        }

        # Calculate consistency (lower std dev = more consistent)
        if len(non_zero_counts) > 1:
            import statistics
            mean_count = statistics.mean(non_zero_counts)
            std_dev = statistics.stdev(non_zero_counts)
            chapter_stats["consistency_score"] = round(max(0, 100 - (std_dev / mean_count * 100)), 1) if mean_count > 0 else 0
        elif len(non_zero_counts) == 1:
            chapter_stats["consistency_score"] = 100  # Perfect consistency with 1 data point

        completeness_analysis["chapter_statistics"][chapter] = chapter_stats

        # Flag missing data
        for year in all_years:
            if data_matrix[chapter][year] == 0:
                completeness_analysis["missing_data"].append({
                    "chapter": chapter,
                    "year": year,
                    "expected_range": f"{chapter_stats['min_tables']}-{chapter_stats['max_tables']}" if chapter_stats['max_tables'] > 0 else "unknown"
                })

        # Flag suspiciously low/high extractions
        if non_zero_counts:
            avg = chapter_stats["avg_tables_per_year"]
            for year in all_years:
                count = data_matrix[chapter][year]
                if count > 0:
                    if count < avg * 0.5:  # Less than 50% of average
                        completeness_analysis["low_extraction"].append({
                            "chapter": chapter,
                            "year": year,
                            "count": count,
                            "expected_avg": avg,
                            "deviation_pct": round((avg - count) / avg * 100, 1)
                        })
                    elif count > avg * 2:  # More than 200% of average
                        completeness_analysis["high_extraction"].append({
                            "chapter": chapter,
                            "year": year,
                            "count": count,
                            "expected_avg": avg,
                            "deviation_pct": round((count - avg) / avg * 100, 1)
                        })

    # Analyze each year across chapters
    for year in all_years:
        year_counts = [data_matrix[chapter][year] for chapter in all_chapters]
        non_zero_counts = [c for c in year_counts if c > 0]

        year_stats = {
            "total_tables": sum(year_counts),
            "chapters_with_data": len(non_zero_counts),
            "chapters_missing": len(all_chapters) - len(non_zero_counts),
            "avg_tables_per_chapter": round(sum(non_zero_counts) / len(non_zero_counts), 1) if non_zero_counts else 0,
            "min_tables": min(non_zero_counts) if non_zero_counts else 0,
            "max_tables": max(non_zero_counts) if non_zero_counts else 0
        }

        completeness_analysis["year_statistics"][year] = year_stats

    # Generate visual matrix representation (text-based)
    def create_text_matrix():
        # Create a compact visual representation
        visual_lines = []
        visual_lines.append("📊 TABLE COUNT MATRIX (Chapter × Year)")
        visual_lines.append("=" * 60)

        # Header with years
        header = "Chapter".ljust(12)
        for year in all_years:
            header += f"{str(year)[-2:].rjust(3)}"  # Last 2 digits of year
        visual_lines.append(header)
        visual_lines.append("-" * len(header))

        # Data rows
        for chapter in all_chapters:
            row = chapter[:10].ljust(12)  # Truncate long chapter names
            for year in all_years:
                count = data_matrix[chapter][year]
                if count == 0:
                    row += "  ."
                else:
                    row += f"{count:3d}"
            visual_lines.append(row)

        return "\n".join(visual_lines)

    text_matrix = create_text_matrix()

    # Generate report
    print("\n" + "="*60)
    print("📋 CHAPTER COMPLETENESS OVERVIEW REPORT")
    print("="*60)

    print(f"📊 Data Coverage:")
    print(f"   Chapters: {stats['total_chapters']}")
    print(f"   Years: {stats['total_years']} ({stats['year_range']})")
    print(f"   Total extractions: {stats['total_extractions']}")

    missing_count = len(completeness_analysis["missing_data"])
    low_count = len(completeness_analysis["low_extraction"])
    high_count = len(completeness_analysis["high_extraction"])

    print(f"\n⚠️  Potential Issues:")
    print(f"   Missing data points: {missing_count}")
    print(f"   Low extraction flags: {low_count}")
    print(f"   High extraction flags: {high_count}")

    # Show most consistent chapters
    consistent_chapters = sorted(
        completeness_analysis["chapter_statistics"].items(),
        key=lambda x: x[1]["consistency_score"],
        reverse=True
    )

    print(f"\n✅ Most Consistent Chapters (top 5):")
    for chapter, stats_data in consistent_chapters[:5]:
        print(f"   • {chapter}: {stats_data['consistency_score']}% consistency, " +
              f"avg {stats_data['avg_tables_per_year']} tables/year")

    # Show chapters needing attention
    problem_chapters = sorted(
        completeness_analysis["chapter_statistics"].items(),
        key=lambda x: (x[1]["years_missing"], -x[1]["consistency_score"])
    )

    print(f"\n🚨 Chapters Needing Attention (top 5):")
    for chapter, stats_data in problem_chapters[-5:]:
        issues = []
        if stats_data["years_missing"] > 0:
            issues.append(f"{stats_data['years_missing']} missing years")
        if stats_data["consistency_score"] < 50:
            issues.append(f"low consistency ({stats_data['consistency_score']}%)")

        if issues:
            print(f"   • {chapter}: {', '.join(issues)}")

    # Display text matrix
    print(f"\n{text_matrix}")

    # Save detailed data
    overview_data = {
        "summary": stats,
        "completeness_analysis": completeness_analysis,
        "data_matrix": {chapter: dict(year_data) for chapter, year_data in data_matrix.items()},
        "visual_matrix": text_matrix
    }

    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_completeness_overview.json")
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(overview_data, f, ensure_ascii=False, indent=2)

    # Save CSV matrix for external analysis
    csv_path = os.path.join(tables_dir, "validations", "completeness_matrix.csv")
    df_matrix.to_csv(csv_path, index=False, encoding="utf-8-sig")

    print(f"\n💾 Detailed overview saved to: {report_path}")
    print(f"📊 CSV matrix saved to: {csv_path}")

    # Calculate overall completeness score
    total_possible = len(all_chapters) * len(all_years)
    actual_data_points = total_possible - missing_count
    completeness_score = (actual_data_points / total_possible) * 100 if total_possible > 0 else 0

    print(f"\n📈 OVERALL COMPLETENESS SCORE: {completeness_score:.1f}% ({actual_data_points}/{total_possible})")

    return completeness_score > 80  # Consider success if >80% completeness

if __name__ == "__main__":
    # Run completeness overview
    is_complete = generate_completeness_overview()

    if is_complete:
        print("\n✅ Chapter completeness overview PASSED")
    else:
        print("\n❌ Chapter completeness overview FAILED - significant data gaps detected")

📋 Generating completeness overview for 6732 extracted tables...

📋 CHAPTER COMPLETENESS OVERVIEW REPORT
📊 Data Coverage:
   Chapters: 15
   Years: 24 (2001-2024)
   Total extractions: 6732

⚠️  Potential Issues:
   Missing data points: 44
   Low extraction flags: 39
   High extraction flags: 3

✅ Most Consistent Chapters (top 5):
   • 08: 75.1% consistency, avg 35.7 tables/year
   • 09: 70.8% consistency, avg 19.8 tables/year
   • 10: 69.9% consistency, avg 20.8 tables/year
   • 11: 69.5% consistency, avg 6.3 tables/year
   • 07: 63.7% consistency, avg 26.8 tables/year

🚨 Chapters Needing Attention (top 5):
   • 09: 4 missing years
   • 10: 4 missing years
   • 11: 4 missing years
   • 04: 4 missing years
   • 15: 10 missing years

📊 TABLE COUNT MATRIX (Chapter × Year)
Chapter      01 02 03 04 05 06 07 08 09 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
------------------------------------------------------------------------------------
01           23 19 14 16 12 14 15 20 15 14 14 31  

In [ ]:
import os
import json
from collections import defaultdict, Counter

def generate_high_risk_samples(tables_dir="/content/tables"):
    """
    Consolidate all validation findings into a prioritized manual verification queue.
    Combines results from all previous validation phases to identify highest-risk cases.
    """

    # Load all validation reports
    reports = {}
    report_files = {
        "completeness": "validation_file_completeness.json",
        "integrity": "validation_data_integrity.json",
        "patterns": "validation_content_patterns.json",
        "positions": "validation_position_mapping.json",
        "sequences": "validation_serial_sequences.json",
        "cross_chapter": "validation_cross_chapter_analysis.json",
        "size_distribution": "validation_size_distribution.json",
        "overview": "validation_completeness_overview.json"
    }

    print("🔍 Loading validation reports...")

    for report_name, filename in report_files.items():
        filepath = os.path.join(tables_dir, "validations", filename)
        if os.path.exists(filepath):
            with open(filepath, "r", encoding="utf-8") as f:
                reports[report_name] = json.load(f)
            print(f"   ✅ Loaded {report_name}")
        else:
            print(f"   ⚠️  Missing {report_name} report: {filename}")
            reports[report_name] = {}

    # Initialize risk assessment structure
    risk_assessment = {
        "document_risks": defaultdict(lambda: {
            "risk_score": 0,
            "risk_factors": [],
            "issues": [],
            "priority": "low"
        }),
        "table_risks": defaultdict(lambda: {
            "risk_score": 0,
            "risk_factors": [],
            "issues": [],
            "priority": "low"
        })
    }

    print(f"\n📊 Analyzing risk factors from {len(reports)} validation reports...")

    # Process completeness issues (missing files)
    if "completeness" in reports and "missing_files" in reports["completeness"]:
        for missing in reports["completeness"]["missing_files"]:
            identifier = missing["identifier"]
            parts = identifier.split("_")
            if len(parts) == 3:
                doc_key = f"{parts[1]}_{parts[2]}"  # chapter_year

                risk_assessment["document_risks"][doc_key]["risk_score"] += 10
                risk_assessment["document_risks"][doc_key]["risk_factors"].append("missing_files")
                risk_assessment["document_risks"][doc_key]["issues"].append({
                    "type": "missing_file",
                    "identifier": identifier,
                    "details": missing["table_name"][:50]
                })

    # Process integrity issues
    if "integrity" in reports and "issues" in reports["integrity"]:
        for issue_type, issue_list in reports["integrity"]["issues"].items():
            for issue in issue_list:
                identifier = issue["identifier"]
                parts = identifier.split("_")
                if len(parts) == 3:
                    doc_key = f"{parts[1]}_{parts[2]}"

                    # Weight different integrity issues
                    weights = {
                        "empty_files": 8,
                        "truncated_tables": 6,
                        "encoding_errors": 5,
                        "missing_luach": 7,
                        "malformed_csv": 5,
                        "suspicious_size": 3
                    }

                    weight = weights.get(issue_type, 3)
                    risk_assessment["table_risks"][identifier]["risk_score"] += weight
                    risk_assessment["table_risks"][identifier]["risk_factors"].append(issue_type)
                    risk_assessment["table_risks"][identifier]["issues"].append({
                        "type": issue_type,
                        "details": issue
                    })

                    # Also flag document
                    risk_assessment["document_risks"][doc_key]["risk_score"] += weight // 2
                    if issue_type not in risk_assessment["document_risks"][doc_key]["risk_factors"]:
                        risk_assessment["document_risks"][doc_key]["risk_factors"].append(f"table_{issue_type}")

    # Process pattern issues
    if "patterns" in reports and "issues" in reports["patterns"]:
        for issue_type, issue_list in reports["patterns"]["issues"].items():
            for issue in issue_list:
                identifier = issue["identifier"]
                parts = identifier.split("_")
                if len(parts) == 3:
                    doc_key = f"{parts[1]}_{parts[2]}"

                    # Weight pattern issues
                    weights = {
                        "no_hebrew_content": 9,
                        "all_empty_cells": 8,
                        "no_table_structure": 6,
                        "suspicious_content": 4
                    }

                    weight = weights.get(issue_type, 3)
                    risk_assessment["table_risks"][identifier]["risk_score"] += weight
                    risk_assessment["table_risks"][identifier]["risk_factors"].append(issue_type)
                    risk_assessment["table_risks"][identifier]["issues"].append({
                        "type": issue_type,
                        "details": issue
                    })

    # Process position mapping gaps
    if "positions" in reports and "gap_analysis" in reports["positions"]:
        for gap_doc in reports["positions"]["gap_analysis"].get("documents_with_gaps", []):
            doc_key = gap_doc["document"]
            gap_analysis = gap_doc["gap_analysis"]

            # Weight based on severity of gaps
            missing_count = gap_analysis["missing_count"]
            extra_count = gap_analysis["extra_count"]

            risk_score = missing_count * 5 + extra_count * 3
            risk_assessment["document_risks"][doc_key]["risk_score"] += risk_score
            risk_assessment["document_risks"][doc_key]["risk_factors"].append("extraction_gaps")
            risk_assessment["document_risks"][doc_key]["issues"].append({
                "type": "extraction_gaps",
                "missing_serials": gap_doc.get("missing_serials", []),
                "extra_serials": gap_doc.get("extra_serials", []),
                "details": gap_analysis
            })

    # Process sequence issues
    if "sequences" in reports and "issues" in reports["sequences"]:
        for issue_type, issue_list in reports["sequences"]["issues"].items():
            for issue in issue_list:
                doc_key = issue["document"]

                # Weight sequence issues
                weights = {
                    "missing_serials": 7,
                    "duplicate_serials": 8,
                    "non_consecutive": 5,
                    "starting_issues": 4
                }

                weight = weights.get(issue_type, 3)
                risk_assessment["document_risks"][doc_key]["risk_score"] += weight
                risk_assessment["document_risks"][doc_key]["risk_factors"].append(issue_type)
                risk_assessment["document_risks"][doc_key]["issues"].append({
                    "type": issue_type,
                    "details": issue
                })

    # Process cross-chapter anomalies
    if "cross_chapter" in reports and "flagged_chapters" in reports["cross_chapter"]:
        for flagged in reports["cross_chapter"]["flagged_chapters"]:
            chapter = flagged["chapter"]
            severity = flagged["severity"]

            for outlier in flagged["outliers"]:
                year = outlier["year"]
                doc_key = f"{chapter}_{year}"

                # Weight by severity and deviation
                weight = 8 if severity == "high" else 5
                deviation = outlier.get("deviation_percent", 0)
                if deviation > 75:
                    weight += 3

                risk_assessment["document_risks"][doc_key]["risk_score"] += weight
                risk_assessment["document_risks"][doc_key]["risk_factors"].append("cross_chapter_anomaly")
                risk_assessment["document_risks"][doc_key]["issues"].append({
                    "type": "cross_chapter_anomaly",
                    "severity": severity,
                    "details": outlier
                })

    # Process size distribution anomalies
    if "size_distribution" in reports and "size_issues" in reports["size_distribution"]:
        for issue_type, issue_list in reports["size_distribution"]["size_issues"].items():
            for issue in issue_list:
                identifier = issue["identifier"]
                parts = identifier.split("_")
                if len(parts) == 3:
                    doc_key = f"{parts[1]}_{parts[2]}"

                    # Weight size issues
                    weights = {
                        "too_small": 6,
                        "extremely_large": 5,
                        "unreadable": 8,
                        "suspicious_dimensions": 3
                    }

                    weight = weights.get(issue_type, 3)
                    risk_assessment["table_risks"][identifier]["risk_score"] += weight
                    risk_assessment["table_risks"][identifier]["risk_factors"].append(issue_type)
                    risk_assessment["table_risks"][identifier]["issues"].append({
                        "type": issue_type,
                        "details": issue
                    })

    # Assign priority levels based on risk scores
    def assign_priority(risk_score):
        if risk_score >= 20:
            return "critical"
        elif risk_score >= 15:
            return "high"
        elif risk_score >= 8:
            return "medium"
        else:
            return "low"

    # Finalize risk assessment and prioritization
    for doc_key, risk_data in risk_assessment["document_risks"].items():
        risk_data["priority"] = assign_priority(risk_data["risk_score"])

    for table_id, risk_data in risk_assessment["table_risks"].items():
        risk_data["priority"] = assign_priority(risk_data["risk_score"])

    # Generate prioritized verification queues
    doc_queue = sorted(
        [(doc_key, data) for doc_key, data in risk_assessment["document_risks"].items()],
        key=lambda x: (-x[1]["risk_score"], x[0])
    )

    table_queue = sorted(
        [(table_id, data) for table_id, data in risk_assessment["table_risks"].items()],
        key=lambda x: (-x[1]["risk_score"], x[0])
    )

    # Generate summary statistics
    priority_counts = {
        "documents": Counter(data["priority"] for _, data in doc_queue),
        "tables": Counter(data["priority"] for _, data in table_queue)
    }

    # Generate report
    print("\n" + "="*60)
    print("🎯 HIGH-RISK SAMPLE GENERATOR REPORT")
    print("="*60)

    print(f"📊 Risk Assessment Summary:")
    print(f"   Documents flagged: {len(doc_queue)}")
    print(f"   Tables flagged: {len(table_queue)}")

    print(f"\n🚨 Priority Distribution:")
    print(f"   Documents - Critical: {priority_counts['documents']['critical']}, " +
          f"High: {priority_counts['documents']['high']}, " +
          f"Medium: {priority_counts['documents']['medium']}, " +
          f"Low: {priority_counts['documents']['low']}")
    print(f"   Tables - Critical: {priority_counts['tables']['critical']}, " +
          f"High: {priority_counts['tables']['high']}, " +
          f"Medium: {priority_counts['tables']['medium']}, " +
          f"Low: {priority_counts['tables']['low']}")

    print(f"\n🎯 TOP PRIORITY DOCUMENTS FOR MANUAL VERIFICATION:")
    for i, (doc_key, risk_data) in enumerate(doc_queue[:10]):
        factors = ", ".join(risk_data["risk_factors"][:3])
        if len(risk_data["risk_factors"]) > 3:
            factors += f" (+{len(risk_data['risk_factors'])-3} more)"
        print(f"   {i+1}. {doc_key} (Score: {risk_data['risk_score']}, Priority: {risk_data['priority']})")
        print(f"      Issues: {factors}")

    print(f"\n🔍 TOP PRIORITY TABLES FOR DETAILED REVIEW:")
    for i, (table_id, risk_data) in enumerate(table_queue[:5]):
        factors = ", ".join(risk_data["risk_factors"])
        print(f"   {i+1}. {table_id} (Score: {risk_data['risk_score']}, Priority: {risk_data['priority']})")
        print(f"      Issues: {factors}")

    # Generate actionable verification lists
    verification_actions = {
        "immediate_action": [],  # Critical priority items
        "high_priority": [],     # High priority items
        "routine_check": []      # Medium priority items
    }

    for doc_key, risk_data in doc_queue:
        action_item = {
            "document": doc_key,
            "risk_score": risk_data["risk_score"],
            "issues": risk_data["issues"],
            "recommended_action": "Compare extraction count against TOC manually",
            "risk_factors": risk_data["risk_factors"]
        }

        if risk_data["priority"] == "critical":
            verification_actions["immediate_action"].append(action_item)
        elif risk_data["priority"] == "high":
            verification_actions["high_priority"].append(action_item)
        elif risk_data["priority"] == "medium":
            verification_actions["routine_check"].append(action_item)

    # Save comprehensive risk assessment
    risk_report = {
        "summary": {
            "documents_flagged": len(doc_queue),
            "tables_flagged": len(table_queue),
            "priority_distribution": priority_counts
        },
        "verification_actions": verification_actions,
        "document_risks": dict(risk_assessment["document_risks"]),
        "table_risks": dict(risk_assessment["table_risks"]),
        "prioritized_queues": {
            "documents": [(doc, data) for doc, data in doc_queue],
            "tables": [(table, data) for table, data in table_queue]
        }
    }

    if not os.path.exists(tables_dir + "/validations/"):
      os.mkdir(tables_dir + "/validations/")
    report_path = os.path.join(tables_dir, "validations", "validation_high_risk_samples.json")
    with open(report_path, "w", encoding="utf-8") as f:
        json.dump(risk_report, f, ensure_ascii=False, indent=2)

    # Generate simplified action list for manual verification
    action_list = []
    for action in verification_actions["immediate_action"] + verification_actions["high_priority"]:
        action_list.append({
            "document": action["document"],
            "priority": "IMMEDIATE" if action in verification_actions["immediate_action"] else "HIGH",
            "risk_score": action["risk_score"],
            "key_issues": action["risk_factors"][:3],
            "action": "Manual TOC verification required"
        })

    action_path = os.path.join(tables_dir, "validations", "manual_verification_actions.json")
    with open(action_path, "w", encoding="utf-8") as f:
        json.dump(action_list, f, ensure_ascii=False, indent=2)

    print(f"\n💾 Risk assessment saved to: {report_path}")
    print(f"📋 Action list saved to: {action_path}")
    print(f"\n📈 RECOMMENDED MANUAL VERIFICATION:")
    print(f"   Immediate action needed: {len(verification_actions['immediate_action'])} documents")
    print(f"   High priority review: {len(verification_actions['high_priority'])} documents")
    print(f"   Routine verification: {len(verification_actions['routine_check'])} documents")

    # Success criteria: reasonable number of high-risk items
    total_high_risk = len(verification_actions['immediate_action']) + len(verification_actions['high_priority'])
    total_docs = len(risk_assessment["document_risks"])

    return total_high_risk < (total_docs * 0.2)  # Success if <20% are high-risk

if __name__ == "__main__":
    # Run high-risk sample generation
    is_manageable = generate_high_risk_samples()

    if is_manageable:
        print("\n✅ High-risk sample generation PASSED - manageable number of issues")
    else:
        print("\n❌ High-risk sample generation FAILED - many high-priority issues detected")

🔍 Loading validation reports...
   ✅ Loaded completeness
   ✅ Loaded integrity
   ✅ Loaded patterns
   ⚠️  Missing positions report: validation_position_mapping.json
   ✅ Loaded sequences
   ✅ Loaded cross_chapter
   ✅ Loaded size_distribution
   ✅ Loaded overview

📊 Analyzing risk factors from 8 validation reports...

🎯 HIGH-RISK SAMPLE GENERATOR REPORT
📊 Risk Assessment Summary:
   Documents flagged: 168
   Tables flagged: 2168

🚨 Priority Distribution:
   Documents - Critical: 3, High: 4, Medium: 80, Low: 81
   Tables - Critical: 0, High: 196, Medium: 83, Low: 1889

🎯 TOP PRIORITY DOCUMENTS FOR MANUAL VERIFICATION:
   1. 01_2012 (Score: 29, Priority: critical)
      Issues: table_truncated_tables, table_truncated_tables, table_truncated_tables (+6 more)
   2. 07_2011 (Score: 25, Priority: critical)
      Issues: table_truncated_tables, table_truncated_tables, table_truncated_tables (+6 more)
   3. 07_2019 (Score: 24, Priority: critical)
      Issues: table_truncated_tables, table_tr

In [ ]:
# Uploads content/tables/ folder to drive
import os
from googleapiclient.http import MediaFileUpload

target_folder_id = '1EhFlIijSkO9HzTH6ymdFyE5_wGouS63z'
local_file_path = "/content/tables"

def upload_files_to_drive(local_path, target_folder_id):
    """
    Upload a single file or entire folder structure to a specified Google Drive folder.

    Args:
        local_path (str): Path to the local file or folder to upload
        target_folder_id (str): Drive folder ID where the content should be uploaded

    Returns:
        bool: True if all uploads successful, False otherwise
    """
    # Handle single file
    if os.path.isfile(local_path):
        try:
            # Extract filename from local path
            filename = os.path.basename(local_path)

            # Fix duplicate chapter numbers (e.g., "09_09.docx" -> "09.docx")
            if "_" in filename and filename.endswith(".docx"):
                parts = filename.replace(".docx", "").split("_")
                if len(parts) == 2 and parts[0] == parts[1]:
                    filename = f"{parts[0]}.docx"

            # Set up file metadata
            file_metadata = {
                'name': filename,
                'parents': [target_folder_id]
            }

            # Create media upload object
            media = MediaFileUpload(
                local_path,
                mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document'
            )

            # Upload the file
            uploaded_file = drive_service.files().create(
                body=file_metadata,
                media_body=media,
                fields='id'
            ).execute()

            file_id = uploaded_file.get('id')
            print(f"✅ Successfully uploaded {filename} (ID: {file_id})")
            return True

        except Exception as e:
            print(f"❌ Failed to upload {local_path}: {e}")
            return False

    # Handle folder using os.walk
    elif os.path.isdir(local_path):
        try:
            folder_map = {}  # Maps local folder paths to Drive folder IDs
            folder_map[local_path] = target_folder_id  # Root folder maps to target
            success_count = 0
            total_count = 0

            for root, dirs, files in os.walk(local_path):
                # Create folders first
                for dir_name in dirs:
                    total_count += 1
                    try:
                        current_dir_path = os.path.join(root, dir_name)
                        parent_folder_id = folder_map[root]

                        # Create folder in Drive
                        folder_metadata = {
                            'name': dir_name,
                            'mimeType': 'application/vnd.google-apps.folder',
                            'parents': [parent_folder_id]
                        }

                        created_folder = drive_service.files().create(
                            body=folder_metadata,
                            fields='id'
                        ).execute()

                        folder_id = created_folder.get('id')
                        folder_map[current_dir_path] = folder_id
                        print(f"📁 Created folder: {dir_name} (ID: {folder_id})")
                        success_count += 1

                    except Exception as e:
                        print(f"❌ Failed to create folder {dir_name}: {e}")

                # Upload files in current directory
                for file_name in files:
                    total_count += 1
                    try:
                        file_path = os.path.join(root, file_name)
                        parent_folder_id = folder_map[root]

                        # Fix duplicate chapter numbers (e.g., "09_09.docx" -> "09.docx")
                        display_name = file_name
                        if "_" in file_name and file_name.endswith(".docx"):
                            parts = file_name.replace(".docx", "").split("_")
                            if len(parts) == 2 and parts[0] == parts[1]:
                                display_name = f"{parts[0]}.docx"

                        # Set up file metadata
                        file_metadata = {
                            'name': display_name,
                            'parents': [parent_folder_id]
                        }

                        # Create media upload object
                        media = MediaFileUpload(
                            file_path,
                            mimetype='application/vnd.openxmlformats-officedocument.wordprocessingml.document'
                        )

                        # Upload the file
                        uploaded_file = drive_service.files().create(
                            body=file_metadata,
                            media_body=media,
                            fields='id'
                        ).execute()

                        file_id = uploaded_file.get('id')
                        print(f"✅ Successfully uploaded {display_name} (ID: {file_id})")
                        success_count += 1

                    except Exception as e:
                        print(f"❌ Failed to upload {file_name}: {e}")

            print(f"📊 Upload summary: {success_count}/{total_count} items uploaded successfully")
            return success_count == total_count

        except Exception as e:
            print(f"❌ Failed to process folder {local_path}: {e}")
            return False

    else:
        print(f"❌ Path does not exist: {local_path}")
        return False

upload_files_to_drive(local_file_path, target_folder_id)

📁 Created folder: 2004 (ID: 11UCCKzQ8axHtXYemrvuO-4zWXG4hcQqd)
📁 Created folder: 2008 (ID: 1SJ38aDr28ZdPtbvfbAOeXY2Uo7vBT5Qs)
📁 Created folder: 2016 (ID: 1RcXAQRP_n8gSiwgbyUQS1QRHFK-cxk33)
📁 Created folder: .ipynb_checkpoints (ID: 1Gb0eN7flJ0VA26PlZ7P_PcDh-c67kUy1)
📁 Created folder: 2011 (ID: 1zfAF8xYPMU8MkS0WojpNkBDqe8pka-Qs)
📁 Created folder: 2013 (ID: 1Vag8zU5tWCRQiWp_cInqZRsC8zbW0iti)
📁 Created folder: 2007 (ID: 1i_RIo_fAWLc5lRcpBzwinuRGPFMWPMUU)
📁 Created folder: 2001 (ID: 1MVV2piMBIN0PegnctM2B0sraEpGKb46E)
📁 Created folder: 2023 (ID: 15CNxKks2X1W0N7aMQfOz44BUg4yeGArX)
📁 Created folder: 2015 (ID: 1UoZC4PBxSynO-BZtQ9rDvwasmjv16q3h)
📁 Created folder: 2018 (ID: 17rdS6eeNyZ0xrwkLAp5MmrtRTpA8ZewG)
📁 Created folder: 2021 (ID: 1yeqJGsbgLUHkdZAth_xrKAW0Mo-fy9_l)
📁 Created folder: 2006 (ID: 1yp1gOrQ98kAMbb1vsurtE9iWxIPqpI9g)
📁 Created folder: 2020 (ID: 19rpGF_Czn5r2WzfYEpYPiLCjgB-wEGmv)
📁 Created folder: 2022 (ID: 14qrKXJMNN6B1TN6gEAgcvuz4LotXUL-c)
📁 Created folder: 2019 (ID: 1S9FCrv16xJPK

KeyboardInterrupt: 

In [ ]:
from google.colab import files
import shutil

shutil.make_archive("tables", 'zip', "tables")
files.download("tables.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>